In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import os
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import eli5
import shap
from IPython.display import HTML
import json
import altair as alt

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

In [4]:
# with pd.HDFStore('./hdf/Store.h5',complib='blosc:blosclz') as store:
#     categorial_features = list(store['initial/categorial_features'].values.flatten())
#     numerical_features = list(store['initial/numerical_features'].values.flatten())
categorial_features = json.load(open('pickles/categorial_features.json','r'))
numerical_features = json.load(open('pickles/numerical_features.json','r'))

In [5]:
from BayDS import *


In [6]:
# with pd.HDFStore('./hdf/pre-aggregate.h5') as store:
#     data = store[f'data']


In [108]:
# # Fixing same data timestamps for same card_id
# check_data = data.reset_index().set_index(['card_id','Date'])

# duplicate_transactions = check_data[check_data.index.duplicated()]['TransactionID'].values

# for itid,tid in enumerate(duplicate_transactions):
#     print(itid)
    
#     q = data.loc[tid]
#     date = q['Date']
#     card_id = q['card_id']
#     alldup = data[data['Date']==date]
#     alldup = alldup[alldup['card_id']==card_id]
# #     print(alldup.index)
#     for it,idx in enumerate(alldup.index):
# #         print(idx)
#         data.ix[idx,'Date']+= pd.Timedelta(seconds=it)
# #     print(data.loc[alldup.index])
# #     break

In [109]:
# with pd.HDFStore('./hdf/pre-aggregate-no-duplicates.h5') as store:
#     store[f'data'] = data

In [ ]:
with pd.HDFStore('./hdf/pre-aggregate-no-duplicates.h5') as store:
    data = store[f'data']


In [110]:
# data_slice = data[['card_id','Date','TransactionAmt']]
# q = data_slice.groupby(['card_id']).rolling(2,on='Date', min_periods=1)['TransactionAmt'].sum()

In [158]:
def aggregate_with_time(data, group_column,num_feature, window_size=5):
    data_slice = data[['Date',group_column,num_feature]].reset_index()
    gb = data_slice.groupby([group_column])
    q = gb.rolling(window_size,on='Date', min_periods=1)[num_feature].agg(['mean','std'])
    ds = data_slice.set_index(['card_id','Date'])
    ds = ds.join(q)
    ds = ds.set_index('TransactionID').sort_index()
    to_mean = ds[num_feature] / ds['mean']
    to_std = ds[num_feature] / ds['std']
    df = pd.DataFrame(index=data.index)
    df[f'{num_feature}_by_{group_column}_ws{window_size}_to_mean'] = to_mean
    df[f'{num_feature}_by_{group_column}_ws{window_size}_to_std'] = to_std
    return df
    

In [157]:
aggregate_with_time(data, 'card_id','TransactionAmt')


,TransactionAmt_by_card_id_window_5_to_mean,TransactionAmt_by_card_id_window_5_to_std
TransactionID,,
2987000,1.000000000000000,NaN
2987001,1.000000000000000,NaN
2987002,1.000000000000000,NaN
2987003,1.000000000000000,NaN
2987004,1.000000000000000,NaN
...,...,...
4170235,2.535564853556485,2.866565109310470
4170236,0.602847856368983,1.184392382711580
4170237,0.424656050265410,0.629227222802077


In [154]:
to_std

TransactionID
2987000                  NaN
2987001                  NaN
2987002                  NaN
2987003                  NaN
2987004                  NaN
                 ...        
4170235    2.866565109310470
4170236    1.184392382711580
4170237    0.629227222802077
4170238    2.640522875816993
4170239    0.206657003142942
Length: 1097231, dtype: float64

In [138]:
data_slice['TransactionAmt'] 

0           68.500000
1           29.000000
2           59.000000
3           50.000000
4           50.000000
              ...    
1097226     94.687500
1097227     12.171875
1097228     49.000000
1097229    202.000000
1097230     24.343750
Name: TransactionAmt, Length: 1097231, dtype: float16

In [19]:
q.head()

card_id  Date               
0        2017-12-02 00:30:58         NaN
         2017-12-02 00:35:38    306.0000
         2017-12-02 00:44:45    187.9375
         2017-12-02 00:47:30    215.8750
         2017-12-02 00:59:29    150.9375
Name: TransactionAmt, dtype: float64

In [26]:
data_slice.groupby(['card_id']).rolling(2,on='Date', min_periods=1)['TransactionAmt'].mean()

card_id  Date               
0        2017-12-02 00:30:58    226.00000
         2017-12-02 00:35:38    153.00000
         2017-12-02 00:44:45     93.96875
         2017-12-02 00:47:30    107.93750
         2017-12-02 00:59:29     75.46875
                                  ...    
19224    2018-09-29 14:11:47     78.46875
19225    2017-12-10 20:21:46    226.00000
         2018-01-31 15:58:56    142.50000
         2018-02-17 16:46:32     88.00000
         2018-09-16 00:15:50    226.00000
Name: TransactionAmt, Length: 1097231, dtype: float64

In [13]:
data['TransactionAmt']

TransactionID
2987000     68.500000
2987001     29.000000
2987002     59.000000
2987003     50.000000
2987004     50.000000
              ...    
4170235     94.687500
4170236     12.171875
4170237     49.000000
4170238    202.000000
4170239     24.343750
Name: TransactionAmt, Length: 1097231, dtype: float16

# Temporal Numerical Aggregates

In [ ]:
def add_temporal_numerical_aggregate(df, target_column, group_column):
    to_mean = pd.DataFrame()
    to_mean[f'{target_column}_to_mean_{group_column}'] = df[target_column] / df.groupby([group_column])[target_column].transform('mean')


In [65]:
def add_categorial_aggregate(df, target_column, group_column):
    print(target_column, group_column)
    data_slice = df[[group_column,target_column]]
    gb = data_slice.groupby([group_column,target_column])[target_column].transform('count') /  data_slice.groupby([group_column])[target_column].transform('count')
    return gb
    
#     if scoring:
#         new_score = quick_score2(df,categorial_features,addition=to_mean,categorial_features_in_addition=[])
#         delta_score = new_score - score
#         with open('./hdf/num_agg_scores.csv','a') as f:
#             f.write(f'{target_column}_to_mean_{group_column}, {delta_score}\n')
#     if save:
#         with pd.HDFStore('./hdf/engineering.h5') as store:
#             store[f'numerical/{target_column}_to_std_{group_column}'] = to_mean[[f'{target_column}_to_mean_{group_column}']]

#     to_std = pd.DataFrame()
#     to_std[f'{target_column}_to_std_{group_column}'] = df[target_column] / data.groupby([group_column])[target_column].transform('std')
#     if scoring:
#         new_score = quick_score2(df,categorial_features,addition=to_std,categorial_features_in_addition=[])
#         delta_score = new_score - score
#         with open('./hdf/num_agg_scores.csv','a') as f:
#             f.write(f'{target_column}_to_std_{group_column}, {delta_score}\n')
#     if save:
#         with pd.HDFStore('./hdf/engineering.h5') as store:
#             store[f'numerical/{target_column}_to_std_{group_column}'] = to_std[[f'{target_column}_to_std_{group_column}']]
#     return to_mean.join(to_std)
    
# add_numerical_aggregate(data,'TransactionAmt', 'card_id')
# add_numerical_aggregate(data,'dist1', 'card_id')
# add_numerical_aggregate(data,'_Hours', 'card_id')





In [68]:
cat_aggs = pd.DataFrame(index=data.index)
# started = False
for cat_feature in categorial_features:
#     if not started and num_feature != 'V1':
#         continue
#     started = True
    for group_feature in ['card_id']:
        try:
            cat_aggs[f'{cat_feature}_freq_groupby_{group_feature}']=add_categorial_aggregate(data,cat_feature, group_feature)
        except Exception as ex:
            print('Exception', ex)
            

id_13 card_id
id_14 card_id
id_17 card_id
id_18 card_id
id_19 card_id
id_20 card_id
id_21 card_id
id_22 card_id
id_23 card_id
id_24 card_id
id_25 card_id
id_26 card_id
id_27 card_id
id_30 card_id
Exception "['id_30'] not in index"
id_31 card_id
Exception "['id_31'] not in index"
id_33 card_id
DeviceType card_id
DeviceInfo card_id
P_emaildomain card_id
Exception "['P_emaildomain'] not in index"
R_emaildomain card_id
Exception "['R_emaildomain'] not in index"
card1 card_id
card2 card_id
card3 card_id
card5 card_id
addr1 card_id
addr2 card_id
P_emaildomain_bin card_id
P_emaildomain_suffix card_id
R_emaildomain_bin card_id
R_emaildomain_suffix card_id
M1 card_id
Exception "['M1'] not in index"
M2 card_id
M3 card_id
M4 card_id
M5 card_id
M6 card_id
M7 card_id
M8 card_id
M9 card_id
card4 card_id
card6 card_id
ProductCD card_id
id_12 card_id
id_15 card_id
id_16 card_id
id_28 card_id
id_29 card_id
id_32 card_id
id_34 card_id
id_35 card_id
id_36 card_id
id_37 card_id
id_38 card_id


In [69]:
cat_aggs.head()

,id_13_freq_groupby_card_id,id_14_freq_groupby_card_id,id_17_freq_groupby_card_id,id_18_freq_groupby_card_id,id_19_freq_groupby_card_id,id_20_freq_groupby_card_id,id_21_freq_groupby_card_id,id_22_freq_groupby_card_id,id_23_freq_groupby_card_id,id_24_freq_groupby_card_id,...,id_15_freq_groupby_card_id,id_16_freq_groupby_card_id,id_28_freq_groupby_card_id,id_29_freq_groupby_card_id,id_32_freq_groupby_card_id,id_34_freq_groupby_card_id,id_35_freq_groupby_card_id,id_36_freq_groupby_card_id,id_37_freq_groupby_card_id,id_38_freq_groupby_card_id
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,...,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000
2987001,0.929216867469880,0.914909638554217,0.914156626506024,0.966867469879518,0.914156626506024,0.914156626506024,0.993222891566265,0.993222891566265,0.993222891566265,0.993222891566265,...,0.914156626506024,0.914156626506024,0.914156626506024,0.914156626506024,0.915662650602410,0.915662650602410,0.914156626506024,0.914156626506024,0.914156626506024,0.914156626506024
2987002,0.996082820369334,0.994963626189144,0.994963626189144,0.998880805819810,0.994963626189144,0.994963626189144,0.999440402909905,0.999440402909905,0.999440402909905,0.999440402909905,...,0.994404029099049,0.994963626189144,0.994404029099049,0.994404029099049,0.994963626189144,0.994404029099049,0.994404029099049,0.994404029099049,0.994404029099049,0.994404029099049
2987003,0.947908445146014,0.919757958431992,0.920021047092870,0.969481715338069,0.920021047092870,0.920021047092870,0.995527492765062,0.995527492765062,0.995527492765062,0.995527492765062,...,0.918047882136280,0.919363325440674,0.918179426466719,0.918179426466719,0.919757958431992,0.921468034727703,0.918047882136280,0.918047882136280,0.918047882136280,0.918047882136280
2987004,0.833333333333333,0.066666666666667,0.333333333333333,0.900000000000000,0.066666666666667,0.033333333333333,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,...,0.166666666666667,0.166666666666667,0.133333333333333,0.133333333333333,0.066666666666667,0.333333333333333,0.333333333333333,0.266666666666667,0.266666666666667,0.300000000000000


In [70]:
cat_aggs = cat_aggs.astype(np.float32)

In [71]:
with pd.HDFStore('./hdf/categorial_aggregations.h5',complib='blosc:blosclz') as store:
    store.put('cat_aggs', cat_aggs)
    

0) Treat categorial features accurately, find very diverse and unite them

1) Убрать коррелирующие фичи

2) Убрать ту из них что даёт меньший джини

3) TimeKFold instead of StratifiedKFold